In [1]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from nltk.tokenize.toktok import ToktokTokenizer
import en_core_web_sm
from pattern.en import suggest
import pandas as pd

In [2]:
#nltk.download('stopwords')
#nltk.download('punkt')
nlp = spacy.load('en_core_web_sm', parse=True, tag=True, entity=True)
tokenizer = ToktokTokenizer()

In [3]:
#file = open("A Boy's Will - Robert Frost/1.txt", "r")

In [4]:
#txt = file.read()

In [5]:
#list_new = init_process(txt)

In [3]:
stop_words = set(stopwords.words('english'))

In [4]:
def init_process(txt):
    no_new = re.sub('\n', ' ', txt)
    no_spl = re.sub('ñ', ' ', no_new)
    first_parse = re.sub(r'[^\w]', ' ', no_spl)
    #word_tokens = word_tokenize(first_parse)
    #filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    return first_parse

In [5]:
def stemming(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [6]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [7]:
def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [8]:
def correct_spelling(w):
    word_wlf = reduce_lengthening(w)
    correct_word = suggest(word_wlf)
    return correct_word[0][0]

In [9]:
def spelling_correction(words):
    correct = [correct_spelling(w) for w in words]
    return correct

In [10]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stop_words]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    #filtered_text = ' '.join(filtered_tokens)    
    return filtered_tokens

In [11]:
def process(file_name):
    file = open(file_name, "r")
    read_txt = file.read()
    list_new = init_process(read_txt)
    stemmed = stemming(list_new)
    lemma = lemmatize_text(stemmed)
    new_words = remove_stopwords(lemma)
    final = spelling_correction(new_words)
    return final

In [12]:
def refine_text():
    for i in range(1,51):
        new_text_file = str(i)+".txt"
        file_name = "poems/"+new_text_file
        refined_list = process(file_name)
        refined_text = ' '.join(refined_list)
        text_file = open(("refined/"+new_text_file), "w")
        text_file.write(refined_text)
        text_file.close()
    return

In [13]:
#refine_text() # Dont refine everytime

In [13]:
dictionary = {}
for i in range(1,51):
    file_name = ("refined/"+str(i)+".txt")
    text = open(file_name, "r").read()
    tokens = tokenizer.tokenize(text)
    for t in tokens:
        if t in dictionary:
            dictionary.get(t).append(i)
            dictionary[t] = list(set(dictionary.get(t)))
        else:
            dictionary[t] = [1]

In [39]:
# Dont do everytime
# text_file = open("inverted_list.csv", "w")
# text_file.write("Words, Inverted Index \n")
# for item in dictionary.keys():
#     join_items = ", ".join(str(d) for d in dictionary.get(item))
#     text = item+", "+str(join_items)
#     text_file.write(text+" \n")
# text_file.close()

In [55]:
### Code to make inverted index - not working
# import InvertedIndex
# import InvertedIndexQuery

# i = InvertedIndex.Index()

# filename = '1.txt'
# file_to_index = open(filename).read() 
# document_key = filename

#     # index the document, using document_key as the document's
#     # id.
# i.index(file_to_index, document_key)

# filename = '2.txt'
# file_to_index = open(filename).read()
# document_key = filename

# i.index(file_to_index, document_key)

# search_results = InvertedIndexQuery.query('Python and spam', i)
# search_results.sort()

# cnt = 0
# for document in search_results:
#     cnt = cnt + 1
#     print ('%d) %s'.format(cnt, document[1])) 